In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt
import optuna
from vacances_scolaires_france import SchoolHolidayDates
from sklearn.model_selection import train_test_split
import importlib

from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer

from preprocess_FE import (
    get_zone_c_holidays,
    encode_dates,
    engineer_weather_features, 
    remove_outliers, 
    get_public_holidays
)

In [2]:
school_holidays = get_zone_c_holidays()
public_holidays = get_public_holidays()

In [3]:
data = pd.read_parquet(Path("data") / "train.parquet")
data = remove_outliers(data)

important_columns = ["date", "pres", "ff", "t", "vv", "rr1"]
weather_data = pd.read_csv("./external_data/external_data.csv", usecols=important_columns)

/Users/martynaharasym/bike_counters/preprocess_FE.py:217: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby(["counter_name", "date_truncated"])


In [4]:
weather_data["date"] = pd.to_datetime(weather_data["date"])
weather_data = weather_data.dropna(axis=1, how="all")
weather_data.set_index("date", inplace=True)
weather_data = weather_data[~weather_data.index.duplicated(keep="first")]
weather_data_interpolated = weather_data.resample("h").interpolate(method="linear")

In [5]:
columns = ['date', 'nouveaux_patients_hospitalises']
covid_data = pd.read_csv('./synthese-fra (1).csv', parse_dates=False, usecols=columns)
covid_data['date_only'] = pd.to_datetime(covid_data['date']).dt.date

In [6]:
merged_data = data.merge(weather_data_interpolated, on="date", how="left")
merged_data['date_only'] = pd.to_datetime(merged_data['date']).dt.date

merged_data = merged_data.merge(covid_data, on="date_only", how="left")

In [7]:
X = merged_data[["counter_name", "site_name", "date_x", "longitude", "latitude", "ff", "t", "vv", "pres", "rr1", "nouveaux_patients_hospitalises"]]
y = merged_data["log_bike_count"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply date encoding

X_train_encoded1 = encode_dates(X_train, school_holidays, public_holidays)
X_test_encoded1 = encode_dates(X_test, school_holidays, public_holidays)

X_train_encoded3 = engineer_weather_features(X_train_encoded1)
X_test_encoded3 = engineer_weather_features(X_test_encoded1)

# Column transformer for preprocessing
categorical_features = ["counter_name", "site_name", "rain_category", "season"]

numerical_features = list(X_train_encoded3.drop(columns=categorical_features).columns)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numerical_features)
    ]
)

In [8]:
def objective_with_cv(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1.0, 10.0),
    }

    xgb_model = XGBRegressor(**params, random_state=42)
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("model", xgb_model)
    ])

    # Perform cross-validation
    cv_scores = cross_val_score(pipeline, X_train_encoded3, y_train, cv=kf, scoring=rmse_scorer)
    mean_rmse = -cv_scores.mean()  # Negate to get positive RMSE as scores are negative
    return mean_rmse

# Define RMSE as a scorer
rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)

# Create a K-Fold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

study = optuna.create_study(direction="minimize")
study.optimize(objective_with_cv, n_trials=30)

# Best hyperparameters
print("Best hyperparameters:", study.best_params)

# Train the final model with the best hyperparameters using cross-validation
best_params = study.best_params
best_model = XGBRegressor(**best_params, random_state=42)

xgboost_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", best_model)
])

cv_scores_final = cross_val_score(xgboost_pipeline, X_train_encoded3, y_train, cv=kf, scoring=rmse_scorer)
mean_final_rmse = -cv_scores_final.mean()
print(f"Final Cross-Validated RMSE: {mean_final_rmse:.4f}")

[I 2024-12-09 17:04:56,628] A new study created in memory with name: no-name-ba614a9e-1095-4204-95e0-bb11663e811d
[I 2024-12-09 17:05:35,447] Trial 0 finished with value: 0.34688005743320105 and parameters: {'n_estimators': 435, 'max_depth': 9, 'learning_rate': 0.17555723219018465, 'subsample': 0.7875452232618532, 'colsample_bytree': 0.6704957626922161, 'reg_alpha': 2.4107018006660086, 'reg_lambda': 4.996803080669925}. Best is trial 0 with value: 0.34688005743320105.
[I 2024-12-09 17:05:49,840] Trial 1 finished with value: 0.4935410264288199 and parameters: {'n_estimators': 299, 'max_depth': 4, 'learning_rate': 0.1119567068911478, 'subsample': 0.8494954927485123, 'colsample_bytree': 0.8682567506303727, 'reg_alpha': 9.921259617403187, 'reg_lambda': 6.455939184638107}. Best is trial 0 with value: 0.34688005743320105.
[I 2024-12-09 17:06:22,078] Trial 2 finished with value: 0.38561604100135327 and parameters: {'n_estimators': 484, 'max_depth': 7, 'learning_rate': 0.07511359690097413, 'sub

Best hyperparameters: {'n_estimators': 464, 'max_depth': 10, 'learning_rate': 0.1470967494275153, 'subsample': 0.7851262648915907, 'colsample_bytree': 0.9676411355846567, 'reg_alpha': 4.010364799699361, 'reg_lambda': 3.486253794150691}
Final Cross-Validated RMSE: 0.3429


In [9]:
xgboost_pipeline.fit(X_train_encoded3, y_train)
y_pred = xgboost_pipeline.predict(X_test_encoded3)
final_rmse = root_mean_squared_error(y_test, y_pred)
print(f"Final XGBoost RMSE: {final_rmse:.4f}")

Final XGBoost RMSE: 0.3386


In [10]:
df_test = pd.read_parquet("./data/final_test.parquet")
df_test_merged = df_test.merge(weather_data_interpolated, on='date', how='left')
df_test_merged['date_only'] = pd.to_datetime(df_test_merged['date']).dt.date

df_test_merged = df_test_merged.merge(covid_data, on='date_only', how='left')

X = df_test_merged[["counter_name", "site_name", "date_x", "longitude", "latitude", "ff", "t", "vv", "pres", "rr1", "nouveaux_patients_hospitalises"]]

X = encode_dates(X, school_holidays, public_holidays)
X = engineer_weather_features(X)

X_test_final = preprocessor.transform(X)
final_predictions = xgboost_pipeline.named_steps['model'].predict(X_test_final)

results = pd.DataFrame({"Id": np.arange(final_predictions.shape[0]), "log_bike_count": final_predictions})
results.to_csv("submission_xgboost_optuna_cv.csv", index=False)

/Users/martynaharasym/bike_counters/preprocess_FE.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["year"] = X["date_x"].dt.year
/Users/martynaharasym/bike_counters/preprocess_FE.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["month"] = X["date_x"].dt.month
/Users/martynaharasym/bike_counters/preprocess_FE.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat